In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

#train_set, val_set = torch.utils.data.random_split(dataset, [50000, 10000])


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

List of transformations 

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]) #((mean*n_c), (std*n_c))

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download = True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle = True, num_workers = 2)

In [9]:
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 4, shuffle = True, num_workers = 2)

In [8]:
classes = ('plane', 'car', 'bird', 'cat','deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

Visualizing Stuff

In [10]:
import matplotlib.pyplot as plt 
import numpy as np 

In [26]:
def imgshow(img):
    img = (img/2)+0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [27]:
dataiter = iter(trainloader)
img, label = dataiter.next()

In [31]:
img[0].shape

torch.Size([3, 32, 32])

The Model

In [30]:
import torch.nn as nn
import torch.nn.functional as F

In [32]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool1(F.relu(self.conv2(x)))
        
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

net = Net()
    

In [33]:
import torch.optim as optim

In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = .9)

In [40]:
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        inputs, label = data
        
        optimizer.zero_grad()
        
        out = net(inputs)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%2000 == 0:
            print(f'{epoch}, {i} - Loss: {running_loss/2000}')
            running_loss = 0

print("Done")

Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f348b6bb620>
Traceback (most recent call last):
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/aditya/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


0, 0 - Loss: 0.0011482141017913818
0, 2000 - Loss: 1.8415190180540084
0, 4000 - Loss: 1.7689051042497157
0, 6000 - Loss: 1.7207437451779843
0, 8000 - Loss: 1.7079410113692284
0, 10000 - Loss: 1.7022317141294478
0, 12000 - Loss: 1.677804727077484


Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f348b6bb620>
    self._shutdown_workers()
Traceback (most recent call last):
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f348b6bb620>
Traceback (most recent call last):
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
    w.join()
  File "/home/aditya/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/home/aditya/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid

1, 0 - Loss: 0.0008496413230895996
1, 2000 - Loss: 1.6384752558469773
1, 4000 - Loss: 1.642255889505148
1, 6000 - Loss: 1.6363129316717386
1, 8000 - Loss: 1.6121260406970979
1, 10000 - Loss: 1.5869621372520923
1, 12000 - Loss: 1.575574921220541
Done


In [41]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)